# Kaggle Competition: Patfinder Pawpularity score

## 1. Make a model (xgboost) with CNN feautres + Extra factors

Import libraries

In [2]:
import os
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
import warnings
from xgboost import XGBRegressor
from torchvision import transforms
import torch
import torch.nn as nn
import timm
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

#Backgrounds configurations
warnings.filterwarnings("ignore")

Parameters and Hyperparameters

In [4]:
# Path control
path = '../input/petfinder-pawpularity-score/'
img_dir = path + 'train/'
label_path = path + 'train.csv'
path_model = '../models/effnetb0_bs64_lr00001_m09.pt'

# Parameters
random_seed = 17
val_size = 0.2

#Hyperparameters grid
params = {
    'max_depth': [3,6,10], 
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 500, 1000],
    'colsample_bytree': [0.3, 0.7]
}

Setting randomness

In [5]:
def seed_torch(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_torch(seed=random_seed)

Help functions

In [6]:
def rmse(Y,Y_pred):
    return np.sqrt(mean_squared_error(Y, Y_pred))

Instance the dataset class: train and validation instances

In [7]:
df = pd.read_csv(label_path)
df['nPawpularity']=(df['Pawpularity'] - df['Pawpularity'].min()) / (df['Pawpularity'].max() - df['Pawpularity'].min())
df_train, df_val = train_test_split(df, test_size=0.2, random_state=random_seed)
df_train.reset_index(inplace=True)
df_val.reset_index(inplace=True)

Dataset

In [8]:
df_train_ = df_train.drop(columns=['index','Id','nPawpularity'])
X = df_train_.iloc[:,:-1]
Y = df_train_.iloc[:,-1]

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=val_size)

Defining the model and training

In [14]:
# Model class
params_best = {
    'colsample_bytree': 0.7,
    'learning_rate': 0.05,
    'max_depth': 3,
    'n_estimators': 100
}

model = XGBRegressor(random_state=random_seed, booster='gbtree', **params_best) #, tree_method='gpu_hist')
model

XGBRegressor(base_score=None, booster='gbtree', colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=0.7, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.05, max_delta_step=None, max_depth=3,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             random_state=17, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=None)

Grid Search

In [15]:
clf = GridSearchCV(estimator=model, param_grid=params, scoring='neg_mean_squared_error', verbose=1)
clf.fit(X,Y)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


GridSearchCV(estimator=XGBRegressor(base_score=None, booster='gbtree',
                                    colsample_bylevel=None,
                                    colsample_bynode=None, colsample_bytree=0.7,
                                    gamma=None, gpu_id=None,
                                    importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=0.05, max_delta_step=None,
                                    max_depth=3, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=None,
                                    num_parallel_tree=None, random_state=17,
                                    reg_alpha=None, reg_lambda=None,
                                    scale_pos_weight=None, subsample=None,
                                    tree_method=None, validate_parameters=None,
        

In [56]:
position = np.where( clf.cv_results_['rank_test_score'] == min(clf.cv_results_['rank_test_score']) )
clf.cv_results_['params'][position[0][0]]

{'colsample_bytree': 0.7,
 'learning_rate': 0.05,
 'max_depth': 3,
 'n_estimators': 100}

Fit the model

In [58]:
model = clf.best_estimator_
model.fit(X_train, Y_train, early_stopping_rounds=10, eval_set= [(X_train, Y_train), (X_val, Y_val)])

[0]	validation_0-rmse:41.13655	validation_1-rmse:42.01793
[1]	validation_0-rmse:39.60340	validation_1-rmse:40.49027
[2]	validation_0-rmse:38.16687	validation_1-rmse:39.05890
[3]	validation_0-rmse:36.82223	validation_1-rmse:37.71884
[4]	validation_0-rmse:35.56494	validation_1-rmse:36.46687
[5]	validation_0-rmse:34.39082	validation_1-rmse:35.29673
[6]	validation_0-rmse:33.29346	validation_1-rmse:34.20321
[7]	validation_0-rmse:32.27241	validation_1-rmse:33.18603
[8]	validation_0-rmse:31.32208	validation_1-rmse:32.24002
[9]	validation_0-rmse:30.43646	validation_1-rmse:31.35546
[10]	validation_0-rmse:29.61613	validation_1-rmse:30.53734
[11]	validation_0-rmse:28.85539	validation_1-rmse:29.77956
[12]	validation_0-rmse:28.14891	validation_1-rmse:29.07650
[13]	validation_0-rmse:27.49886	validation_1-rmse:28.42665
[14]	validation_0-rmse:26.89712	validation_1-rmse:27.82533
[15]	validation_0-rmse:26.34090	validation_1-rmse:27.26834
[16]	validation_0-rmse:25.82948	validation_1-rmse:26.75663
[17]	va

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=17,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

Evaluation

In [61]:
Y_preds = model.predict(X_val)
rmse_val = rmse(Y_val,Y_preds)
print(f"RMSE val = {rmse_val}")

RMSE val = 21.279552855393643


In [62]:
ix = random.randint(0,len(Y_preds))
ix, Y_preds[ix], Y_val.iloc[ix]

(848, 37.86341, 43)

Getting the feature map of each image from the CNN

Load CNN model

Images paths

In [63]:
df_train['path'] = path + 'train/' + df_train['Id'] + '.jpg'
df_train.head()

,index,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,nPawpularity,path
0,5481,8d39384ff786f101be108a7bcb734907,0,1,1,1,0,0,0,0,0,0,0,0,29,0.282828,../input/petfinder-pawpularity-score/train/8d3...
1,5441,8c0f8cd3c05d5c33cc069b2c0b5b0098,0,1,1,1,0,0,0,0,1,0,0,0,21,0.202020,../input/petfinder-pawpularity-score/train/8c0...
2,69,01a5427a15d9a398116c92724abb6e16,0,1,1,1,0,0,0,0,0,0,0,0,40,0.393939,../input/petfinder-pawpularity-score/train/01a...
3,5119,83ffc3a217289c40605c8bdc21e05939,0,0,1,1,0,0,0,0,0,1,1,1,31,0.303030,../input/petfinder-pawpularity-score/train/83f...
4,2770,475c2ac1aa6eb3e280808d4e56a91e7d,0,1,1,1,0,0,1,0,0,0,0,0,28,0.272727,../input/petfinder-pawpularity-score/train/475...


Preprocessing images

In [64]:
img_size = 224
batch_size = 256
num_workers=0#4
pin_memory=False#True

# To make transformations of the images
transform = transforms.Compose(
                   [
                    transforms.ToTensor(),
                    transforms.Resize((img_size,img_size)),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                   ])

class Dataset_train(torch.utils.data.Dataset):
    def __init__(self, img_dir, dataframe, img_name, transform=None):
        self.img_dir = img_dir
        self.dataframe = dataframe
        self.transform = transform
        self.img_name = img_name
        
    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, ix):
        img_id = self.dataframe[self.img_name][ix] 
        img_path = self.img_dir + img_id + '.jpg'
        X_ix = plt.imread(img_path)
        if self.transform:
            X_ix = self.transform(X_ix)
        return img_id, X_ix

dataset_train = Dataset_train(img_dir, df_train , 'Id', transform=transform)
dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory)        

Device

In [65]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


Load CNN

In [66]:
class Net_pretrained(nn.Module):
    def __init__(self, timm_model_name, target_size, pretrained=True):
        super().__init__()
        self.model = timm.create_model(timm_model_name, pretrained=pretrained)
        self.n_features = self.model.classifier.in_features
        self.model.classifier = nn.Identity()
        self.fc = nn.Linear(self.n_features, target_size)

    def feature(self, image):
        feature = self.model(image)
        return feature

    def forward(self, image):
        feature = self.feature(image)
        output = self.fc(feature)
        return output

MODEL_NAME='tf_efficientnet_b0_ns'
model = Net_pretrained(timm_model_name=MODEL_NAME, target_size=1, pretrained=False)
model.to(device)
model.load_state_dict(torch.load(path_model))
model.eval()

Net_pretrained(
  (model): EfficientNet(
    (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (act1): SiLU(inplace=True)
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): SiLU(inplace=True)
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_r

Try an tensor sample

In [67]:
tensordeprueba = torch.load('../others/tensordeprueba.pt', map_location=torch.device(device))
feature_map = model.feature(tensordeprueba)
feature_map.shape

torch.Size([5, 1280])

Inference over training set

Getting the feature map (not the final prediction) : 1280 features for each image

In [68]:
y_hat_all = []
id_img_all = []
bar = tqdm(dataloader_train)
with torch.no_grad():
    for id_img, batch in bar:
        # get the inputs; data is a list of [inputs]
        X = batch
        X = X.to(device)
        # forward
        y_hat = model.feature(X)
        # accum in a list predictions
        y_hat_all.append(y_hat)
        #accum in a list identifactor of the image to not loss the treace
        id_img_all.append(id_img)

# concat tensors/lists along batch dimension
predictions = torch.cat(y_hat_all, dim=0)
id_img_all = [item for t in id_img_all for item in t]
#to numpy
predictions = predictions.detach().to("cpu").numpy()
predictions.shape

100%|██████████| 31/31 [02:31<00:00,  4.90s/it]


(7929, 1280)

Insert features into dataframe training

In [69]:
id_img_all[0], predictions[0]

('8d39384ff786f101be108a7bcb734907',
 array([-0.04466913, -0.00340137, -0.01552563, ...,  0.5724023 ,
         0.17565875, -0.0266775 ], dtype=float32))

In [70]:
df_feature_map = pd.DataFrame(data=predictions)
df_feature_map['Id'] = pd.DataFrame(id_img_all) 
df_train_stack = pd.merge(df_train, df_feature_map, on=['Id','Id'], how='left')
df_train_stack.head()

,index,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,...,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279
0,5481,8d39384ff786f101be108a7bcb734907,0,1,1,1,0,0,0,0,...,-0.048660,-0.036852,0.501869,0.101538,-0.011447,0.096394,0.109855,0.572402,0.175659,-0.026678
1,5441,8c0f8cd3c05d5c33cc069b2c0b5b0098,0,1,1,1,0,0,0,0,...,-0.059928,-0.048179,0.110863,0.609003,-0.022778,-0.087733,0.048081,-0.116044,0.110561,0.209084
2,69,01a5427a15d9a398116c92724abb6e16,0,1,1,1,0,0,0,0,...,0.090395,0.054929,0.393047,0.301520,0.102754,0.323335,0.140073,-0.057332,0.314771,-0.119351
3,5119,83ffc3a217289c40605c8bdc21e05939,0,0,1,1,0,0,0,0,...,0.323016,0.029486,0.486436,0.054926,0.015337,-0.036464,0.233930,0.110510,0.141319,0.116216
4,2770,475c2ac1aa6eb3e280808d4e56a91e7d,0,1,1,1,0,0,1,0,...,0.146611,-0.000424,0.014192,0.392609,0.202484,0.528141,-0.127530,0.245761,0.152712,0.286858


Build the xgboost again

In [71]:
df_train_stack_ = df_train_stack.drop(columns=['index','Id','nPawpularity','path'])
X = df_train_stack_.drop(columns=['Pawpularity'])
Y = df_train_stack_['Pawpularity']

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=val_size)

In [74]:
# Model class
model = XGBRegressor(random_state=random_seed)
# Parameters grid
FOLDS = 3
params = {
    'max_depth': [3,6], 
    'learning_rate': [0.01, 0.05],
    'n_estimators': [100],
    'colsample_bytree': [0.7]
}

# Grid Search
clf = GridSearchCV(estimator=model, cv=FOLDS, param_grid=params, scoring='neg_mean_squared_error', verbose=10)
clf.fit(X,Y)
clf.best_params_

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV 1/3; 1/8] START colsample_bytree=0.7, learning_rate=0.01, max_depth=3, n_estimators=100
[CV 1/3; 1/8] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, n_estimators=100;, score=-630.165 total time=  19.5s
[CV 2/3; 1/8] START colsample_bytree=0.7, learning_rate=0.01, max_depth=3, n_estimators=100
[CV 2/3; 1/8] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, n_estimators=100;, score=-556.967 total time=  23.3s
[CV 3/3; 1/8] START colsample_bytree=0.7, learning_rate=0.01, max_depth=3, n_estimators=100
[CV 3/3; 1/8] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, n_estimators=100;, score=-587.438 total time=  23.4s
[CV 1/3; 2/8] START colsample_bytree=0.7, learning_rate=0.01, max_depth=3, n_estimators=500
[CV 1/3; 2/8] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, n_estimators=500;, score=-386.732 total time= 1.8min
[CV 2/3; 2/8] START colsample_bytree=0.7, learning_rate=0.01, max_de

KeyboardInterrupt: 

In [25]:
model = clf.best_estimator_
model.fit(X_train, Y_train, early_stopping_rounds=10, eval_set= [(X_train, Y_train), (X_val, Y_val)])

[0]	validation_0-rmse:39.63276	validation_1-rmse:39.20909
[1]	validation_0-rmse:36.60750	validation_1-rmse:36.33772
[2]	validation_0-rmse:33.94630	validation_1-rmse:33.79595
[3]	validation_0-rmse:31.58274	validation_1-rmse:31.61270
[4]	validation_0-rmse:29.50320	validation_1-rmse:29.73606
[5]	validation_0-rmse:27.66912	validation_1-rmse:28.12486
[6]	validation_0-rmse:26.07471	validation_1-rmse:26.72642
[7]	validation_0-rmse:24.67367	validation_1-rmse:25.51941
[8]	validation_0-rmse:23.45226	validation_1-rmse:24.49393
[9]	validation_0-rmse:22.39732	validation_1-rmse:23.64294
[10]	validation_0-rmse:21.47157	validation_1-rmse:22.91329
[11]	validation_0-rmse:20.66878	validation_1-rmse:22.29137
[12]	validation_0-rmse:19.96660	validation_1-rmse:21.77514
[13]	validation_0-rmse:19.35264	validation_1-rmse:21.36615
[14]	validation_0-rmse:18.82533	validation_1-rmse:21.01291
[15]	validation_0-rmse:18.36995	validation_1-rmse:20.70745
[16]	validation_0-rmse:17.96492	validation_1-rmse:20.49008
[17]	va

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=17,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

Evaluation

In [27]:
Y_preds = model.predict(X_val)
rmse_val = rmse(Y_val, Y_preds)
print(f"RMSE val = {rmse_val}")

RMSE val = 19.308776636887973


Save the model

In [28]:
model.save_model('../models/xgboost_effnetb0_plusfactors.json')